In [ ]:
!pip install rasterio

In [ ]:
import gdal
from osgeo import ogr
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import json
import rasterio
from shapely.geometry import LineString, Polygon, shape
import cv2
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = "/content/drive/My Drive/"

if not os.path.exists(drive_path + '/spacenet-data'):
  print("Path created")
  os.makedirs(drive_path + 'spacenet-data')
drive_path = "/content/drive/My Drive/spacenet-data/"

In [ ]:
#installing aws cli
!curl "https://d1vvhvl2y92vvt.cloudfront.net/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip awscliv2.zip
!sudo ./aws/install

In [ ]:
!/usr/local/bin/aws2 configure

In [ ]:
#display path of aws
!/usr/local/bin/aws2 s3 ls s3://spacenet-dataset/spacenet/

In [ ]:
#display path of aws
!/usr/local/bin/aws2 s3 ls s3://spacenet-dataset/spacenet/SN3_roads/train/

In [ ]:
#display path of aws
!/usr/local/bin/aws2 s3 ls s3://spacenet-dataset/spacenet/SN3_roads/train/AOI_2_Vegas/

In [ ]:
#Images and geojson files related to Vegas
!/usr/local/bin/aws2 s3 ls s3://spacenet-dataset/spacenet/SN3_roads/train/AOI_2_Vegas/PAN/

In [ ]:
!/usr/local/bin/aws2 s3 ls s3://spacenet-dataset/spacenet/SN3_roads/train/AOI_2_Vegas/geojson_roads_speed/

In [ ]:
#download tif from AWS
!/usr/local/bin/aws2 s3api get-object --bucket spacenet-dataset \
    --key spacenet/SN3_roads/train/AOI_2_Vegas/PAN/SN3_roads_train_AOI_2_Vegas_PAN_img100.tif \
    --request-payer requester SN3_roads_train_AOI_2_Vegas_PAN_img100.tif

In [ ]:
#download roads gejson from AWS
!/usr/local/bin/aws2 s3api get-object --bucket spacenet-dataset \
    --key spacenet/SN3_roads/train/AOI_2_Vegas/geojson_roads_speed/road_speed_estimates.txt \
    --request-payer requester road_speed_estimates.txt

In [ ]:
#list of names of all image files present in MS directory
data = pd.read_csv(drive_path + 'shanghai_ms_list.txt', header = None)
image_list = data[3]
image_list = image_list.values

In [ ]:
image_path = '/content/drive/My Drive/spacenet-data/data/'
geojson_path = '/content/drive/My Drive/spacenet-data/geojson_roads/'
geojson_speed_path = '/content/drive/My Drive/spacenet-data/geojson_roads_speed/'
mask_path = '/content/drive/My Drive/spacenet-data/mask/'

In [ ]:
#get satellite images
def get_sat_images(region, image_list):
  if not os.path.exists(drive_path+'data'):
    os.makedirs(drive_path+'data')
  count = 0
  for image in image_list:
    count += 1
    print(f"{count} {image}")
    image = image.replace("MS", "PAN")
    cmd = f'/usr/local/bin/aws2 s3api get-object --bucket spacenet-dataset --key spacenet/SN3_roads/train/{region}/MS/{image} --request-payer requester /content/drive/"My Drive"/spacenet-data/data/{image}'
    os.system(cmd)

In [ ]:
#AOI_2_Vegas, AOI_3_Paris, AOI_4_Shanghai, AOI_5_Khartoum
get_sat_images('AOI_4_Shanghai', image_list)

In [ ]:
#download geojson_roads
def get_geojson_roads(region, image_list):
  if not os.path.exists(drive_path+'geojson_roads'):
    os.makedirs(drive_path+'geojson_roads')
  count = 0
  for image in image_list:
    count += 1
    geojson_name = image.replace("MS","geojson_roads").replace("tif", "geojson")
    print(f"{count} {geojson_name}")
    cmd = f'/usr/local/bin/aws2 s3api get-object --bucket spacenet-dataset --key spacenet/SN3_roads/train/{region}/geojson_roads/{geojson_name} --request-payer requester /content/drive/"My Drive"/spacenet-data/geojson_roads/{geojson_name}'
    os.system(cmd)

In [ ]:
#AOI_2_Vegas, AOI_3_Paris, AOI_4_Shanghai, AOI_5_Khartoum
get_geojson_roads('AOI_4_Shanghai', image_list)

In [ ]:
#download geojson_roads_speed
def get_geojson_speed(region, image_list):
  if not os.path.exists(drive_path+'geojson_roads_speed'):
    os.makedirs(drive_path+'geojson_roads_speed')
  count = 0
  for image in image_list:
    count += 1
    geojson_name = image.replace("MS","geojson_roads_speed").replace("tif", "geojson")
    print(f"{count} {geojson_name}")
    cmd = f'/usr/local/bin/aws2 s3api get-object --bucket spacenet-dataset --key spacenet/SN3_roads/train/{region}/geojson_roads_speed/{geojson_name} --request-payer requester /content/drive/"My Drive"/spacenet-data/geojson_roads_speed/{geojson_name}'
    os.system(cmd)

In [ ]:
#AOI_2_Vegas, AOI_3_Paris, AOI_4_Shanghai, AOI_5_Khartoum
get_geojson_speed('AOI_4_Shanghai', image_list)

In [ ]:
def show_raster(raster_src):
  src1 = rasterio.open(raster_src)
  print(src1.shape)
  plt.imshow(src1.read(1), cmap = 'pink')
  plt.plot()

In [ ]:
#this method was used to create mask for train data
def create_mask_road_lane(geojson_file, raster_src):

  srcRas_ds = gdal.Open(raster_src)
  cols = srcRas_ds.RasterXSize
  rows = srcRas_ds.RasterYSize

  mask = np.zeros((rows, cols))
  with open(geojson_file, 'r') as file:
      geo_json = json.load(file)
      with rasterio.open(raster_src) as map_image:
          for road in geo_json['features']:
              polys = list()
              num_of_lanes = int(road['properties']['lane_number'])
              geometry = road['geometry']
              if geometry['type'] == "MultiLineString":
                  temp = list()
                  for lines in geometry['coordinates']:
                      for coords in lines:
                          temp.append(coords)
                  
                  points = np.array(temp)
              else:
                  points = np.array(geometry['coordinates'])

              for point in points:
                  coords = map_image.index(point[0], point[1])
                  polys.append([coords[1], coords[0]])


                  if num_of_lanes == 1 or num_of_lanes == 2:
                    cv2.polylines(mask, np.int32([polys]), False, (255, 0, 0), thickness = 2)
                  elif num_of_lanes == 3 or num_of_lanes == 4:
                    cv2.polylines(mask, np.int32([polys]), False, (255, 0, 0), thickness = 4)
                  elif num_of_lanes == 5 or num_of_lanes == 6:
                    cv2.polylines(mask, np.int32([polys]), False, (255, 0, 0), thickness = 6)

  return mask

In [ ]:
def get_road_color(road_type):
  case = {'1':(0, 255, 64),
          '2':(255, 255, 0),
          '3':(0, 255, 255),
          '4':(255, 0, 0),
          '5':(0, 128, 255),
          '6':(255, 255, 0)
          }
  return case.get(road_type)

In [ ]:
def create_mask_road_type(geojson_file, raster_src):

  srcRas_ds = gdal.Open(raster_src)
  cols = srcRas_ds.RasterXSize
  rows = srcRas_ds.RasterYSize

  mask = np.zeros((rows, cols))
  with open(geojson_file, 'r') as file:
      geo_json = json.load(file)
      with rasterio.open(raster_src) as map_image:
          for road in geo_json['features']:
              polys = list()
              road_type = road['properties']['road_type']
              geometry = road['geometry']
              if geometry['type'] == "MultiLineString":
                  points = np.array(geometry['coordinates'][0])

              else:
                  points = np.array(geometry['coordinates'])

              for point in points:
                  coords = map_image.index(point[0], point[1])
                  polys.append([coords[1], coords[0]])
                  cv2.polylines(mask, np.int32([polys]), False, get_road_color(road_type), thickness = 3)

  return mask

In [ ]:
#sample data
image = image_path + 'SN3_roads_train_AOI_2_Vegas_MS_img100.tif'
geojson = geojson_path + 'SN3_roads_train_AOI_2_Vegas_geojson_roads_img100.geojson'
geojson_speed = geojson_speed_path + 'SN3_roads_train_AOI_2_Vegas_geojson_roads_speed_img100.geojson'

In [ ]:
show_raster(image)

In [ ]:
mask = create_mask(geojson, image)
plt.imshow(mask)

In [ ]:
mask_lane = create_mask_road_lane(geojson_speed,image)
plt.imshow(mask_lane)

In [ ]:
mask_type = create_mask_road_type(geojson, image)
plt.imshow(mask_type)

In [ ]:
#create road lane mask
count = 0
missing = list()
image_path = '/content/drive/My Drive/spacenet-data/data/'
mask_path = '/content/drive/My Drive/spacenet-data/mask_lane/'
for image_name in image_list:
  try:
    geojson_name = image_name.replace("MS", "geojson_roads_speed").replace("tif", "geojson")
    mask = create_mask_road_lane(geojson_speed_path + geojson_name, image_path + image_name)
    mask_name = image_name.replace("MS", "mask_width").replace("tif", "npy")
    print(f"{count} {image_name}")
    np.save(mask_path+mask_name, mask)
    count += 1
  except Exception as e:
    print("Error "+str(e))
    missing.append(image_name)
    pass
 

In [ ]:
def display_mask(image_name):
    fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize = (10, 10))
    raster = rasterio.open('/content/drive/My Drive/spacenet-data/data/' + image_name)
    ax1.imshow(raster.read(1), cmap = 'pink')
    ax1.set_title('Raster Image')
    ax2.imshow(np.load('/content/drive/My Drive/spacenet-data/mask_lane/' + image_name.replace("MS", "mask").replace(".tif", ".npy")))
    ax2.set_title('Mask')
    plt.show()

In [ ]:
for image in image_list:
  display_mask(image)